# Range Accrual Basics

This notebook demonstrates how to access and use the functionalities of **Range Accrual** within LSEG Financial Analytics SDK.

**You will be able to:**
* Define Range Accrual instrument with basic parameters
* Configure pricing parameters
* Evaluate Range Accrual Analytics (Description, Price and Sensitives)

A range accrual swap is a derivative contract that allows two parties to exchange cash flows based on the performance of an underlying asset within a specified range, typically used to manage interest rate risk.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Range Accrual, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

range_accrual_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "RangeAccrual",
    inputs = [
        sp.NameTypeValue(name="StartDate", type = "date", value=dt.date(2025, 9, 15)),
        sp.NameTypeValue(name="EndDate", type = "date", value= dt.date(2035, 9, 15)),
        sp.NameTypeValue(name="Underlying", type = "string", value="EUR"),
        sp.NameTypeValue(name="Notional", type = "string", value="1000000"),
        sp.NameTypeValue(name="Frequency", type = "string", value="SemiAnnual"),
		sp.NameTypeValue(name="ObservationFrequency", type = "string", value="Daily"),
        sp.NameTypeValue(name="IndexTenor", type = "string", value="6M"),
        sp.NameTypeValue(name="IndexRate", type = "string", value="Libor(Underlying,PeriodStart(),IndexTenor)"),
        sp.NameTypeValue(name="UpperBound", type = "string", value="2.75%"),
        sp.NameTypeValue(name="LowerBound", type = "string", value="0.25%"),
        sp.NameTypeValue(name="Coupon", type = "string", value="3.428%"),
        sp.NameTypeValue(name="DayCount", type = "string", value="30/360"),
        
    ],
    payoff_description = [
			[
				"Schedule type",
				"Schedule description",
				"RangeAccrualLeg",
                "Reinitialisation",
				"Price",
				"PricePercent"
			],
			[
				"AllTheTime",
				"FromTo(DateTable(StartDate,EndDate,Frequency),ObservationFrequency)",
				"$n1 = if(abs(IndexRate)>LowerBound and IndexRate<UpperBound, $n1+1, $n1); $n2 =$n2+1",
				"",
				"",
                ""
			],
            [
				"OnSchedule PeriodEnd",
				"DateTable(StartDate, EndDate, Frequency, Daycount)",
				"Coupon*$n1/$n2*InterestTerm()*Notional",
				"$n1 = 0; $n2 = 0",
				"Coupon*$n1/$n2*InterestTerm()*Notional",
                "Coupon*$n1/$n2*InterestTerm()*100"
			],
            [
				"AtDate",
				"EndDate",
				"",
				"",
				"Notional",
				"100"
			]
		]
)
print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")
range_accrual = sp.StructuredProductsDefinitionInstrument(definition = range_accrual_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

range_accrual_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 9, 12),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "EUR",
            underlying_tag = "EUR",
            underlying_currency = "EUR",
            asset_class = "InterestRate",
            model_name= "HullWhite1Factor",
            calibration_list = [
								{
									"StartDate": "2025-09-15",
									"EndDate": "2030-09-15",
									"Frequency": "SemiAnnual",
									"Tenor": "ENDDATE",
									"UserTenor": "",
									"Calendar": "Target",
									"ProductType": "Swaption",
									"Strike": "ATM",
									"CalibrationType": "Bootstrap",
									"Parameter": "Volatility"
								}
                        	]
            )]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function with error handling
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[range_accrual],
        pricing_preferences=range_accrual_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "RangeAccrual",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Price": 771108.686695319,
        "PricePercent": 77.11086867,
        "RangeAccrualLeg": 179449.963016669,
        "Reinitialisation": 0.0
    }
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_range_accrual_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_range_accrual_valuation)

,Field,Value
0,marketValueInDealCcy,771108.686695


### Greeks

In [6]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,-772.092000
1,gammaAmountInDealCcy,0.847235
2,vegaAmountInDealCcy,0.000000
3,thetaAmountInDealCcy,43.021000
